In [12]:
import utils

In [2]:
X, y, X_test = utils.get_and_process_data()

/Users/gregzguegue/Python_Data_Challenge/utils.py:216: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/gregzguegue/Python_Data_Challenge/utils.py:216: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/gregzguegue/Python_Data_Challenge/utils.py:382: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are

In [13]:
pipeline = utils.create_pipeline_TV(X)

In [14]:
pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  TableVectorizer(),
                                                  Index(['counter_id', 'site_id', 'date', 'counter_installation_date',
       'counter_technical_id', 'latitude', 'longitude',
       'Pressure_Tendency_(hPa/3h)', 'Pressure_Tendency_Code',
       'Wind_Direction_(°)', 'Wind_Speed_(m/s)', 'Air_Temperature_(°C)',
       'Dew_Point_Temperature_(°C...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [15]:
submission = utils.test_fit_and_submission(X_test, pipeline)